# Azure OpenAI Service Chat Completion 기초
Chat Completion API는 GPT-35-Turbo 혹은 GPT-4 모델과의 대화 용도로 만든 새 API다. 기본적으로 모델을 사용할 때 이 API를 사용하는 것을 권장한다. 또, 새롭게 발표된 GPT-4 모델을 사용할 수 있는 유일한 방법이기도 하다.

GPT-35-Turbo와 GPT-4 모델은 대화형 인터페이스에 최적화된 모델이다. 이 모델들의 동작은 이전 GPT-3 모델과는 다르다. 이전 모델에서는 **텍스트 입력-텍스트 출력** 방식을 사용했다. 다시 말해, 이전 모델은 프롬프트의 문자열을 입력받아 해당 프롬프트를 완성한 결과를 반환하는 것이었다. 이와 대조적으로 GPT-35-Turbo와 GPT-4 모델은 **대화 입력-메시지 출력**의 형태로 동작한다. 모델은 채팅 방식의 입력을 예상하고 그 방식에 맞게 작성한 메시지를 반환합니다. 이 방식은 멀티턴 대화용으로 설계된 것이지만 채팅 이외의 상황에도 활용할 수 있다.

https://learn.microsoft.com/azure/ai-services/openai/how-to/chatgpt?tabs=python-new&pivots=programming-language-chat-completions

## 사전 준비

이 파이썬 예제를 실행하려면 다음과 같은 환경이 필요하다:

- Azure OpenAI Service를 사용할 수 있는 [승인 완료](https://aka.ms/oai/access)된 Azure 구독
- Azure OpenAI Service에 배포된 GPT-3.5 Turbo / GPT-4 모델
- Azure OpenAI Service 연동 및 모델 정보
  - OpenAI API 키
  - OpenAI GPT-3.5 Turbo / GPT-4 모델의 배포 이름
  - OpenAI API 버전
- Python(이 예제는 버전 3.10.x로 테스트 했다.)

이 예제에서는 Visual Studio Code와 [Jupyter extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter)를 사용한다.


## 패키지 설치


In [ ]:
!pip install openai

In [ ]:
import openai
openai.__version__

## 라이브러리 및 환경변수 불러오기


## Azure OpenAI 설정
Azure OpenAI와 연동을 위해 필요한 정보는 보안을 위해 하드코딩 하지 말고 환경변수나 [dotenv](https://pypi.org/project/python-dotenv/)로 불러오는 것을 권장한다.

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

#os.environ["AZURE_OPENAI_API_KEY"] = "Your OpenAI API Key"
#os.environ["AZURE_OPENAI_ENDPOINT"] = "https://<Your OpenAI Service>.openai.azure.com/"

# 이 변수에는 모델을 배포했을 때 설정한 커스텀 이름을 입력한다.
#AZURE_OPENAI_DEPLOYMENT_NAME = "gpt-35-turbo"

## Chat Completion 호출
GPT-35-Turbo와 GPT-4 모델은 대화 형식 입력에 최적화되어 있다. `messages` 변수는 시스템(`system`), 사용자(`user`), 어시스턴트(`assistant`)로 분류된 역할과 콘텐츠를 가진 딕셔너리 타입 배열로 전달된다.

In [ ]:
import os
from openai import AzureOpenAI

client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-02-01",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
)

In [ ]:
messages= [
    {"role": "system", "content": "너는 한국사 질문을 답변해주는 역사 교수야."},
    {"role": "user", "content": "정중부, 이고와 함께 무신정변을 일으킨 인물의 이름을 알려줘."}
]

response = client.chat.completions.create(
    model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"), # model = "deployment_name"
    messages= messages,
)

print(response.choices[0].message.model_dump_json(indent=2))

### 전체 응답 내용 출력


In [ ]:
print(response.model_dump_json(indent=2))

## 대화 연결하기
Chat Completion API는 **Stateless API**이기 때문에 API가 대화 이력을 가지지 않는다. 따라서 대화 이력을 고려한 답변을 받고 싶을 때는 다음과 같이 과거의 대화를 배열에 담아 요청을 보내야 한다.

- `"role": "assistant"`: API의 답변
- `"role": "user"`: 사용자 입력

In [ ]:
messages= [
    {"role": "system", "content": "너는 한국사 질문을 답변해주는 역사 교수야."},
    {"role": "user", "content": "정중부, 이고와 함께 무신정변을 일으킨 인물의 이름을 알려줘."},
    {"role": "assistant", "content": "정중부, 이고와 함께 무신정변을 일으킨 인물의 이름은 이의방입니다."},
    {"role": "user", "content": "이의방의 출신은?"},
]

response = client.chat.completions.create(
    model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    messages= messages,
    max_tokens=300,
    temperature=0.0,
    top_p=0.9
)

print(response.choices[0].message.model_dump_json(indent=2))

## 시스템 메시지
시스템 메시지（`system`）는 **배열의 맨 앞**에 위치하며 모델에 최초의 지시를 할당한다. 시스템 메시지에는 다음과 같은 다양한 정보를 지정할 수 있다.

- 어시스턴트에 대한 간략한 설명
- 어시스턴트의 성격적 특징
- 어시스턴트가 지켜야 할 절차 및 규칙
- FAQ의 질문 같은 모델이 답변을 생성하기 위해 필요한 데이터나 정보 

기본적인 지시를 포함시키거나 용례에 맞게 시스템 메시지를 커스터마이징할 수 있다. 시스템 메시지는 생략할 수도 있지만 최적화된 결과를 얻으려면 최소한 기본적인 지시는 포함시키는 것을 권장한다.

### 시스템 메시지 프레임워크 및 템플릿 권장사항
https://learn.microsoft.com/azure/ai-services/openai/concepts/system-message

In [ ]:
messages= [
    {"role": "system", "content": "너는 한국사 질문을 답변해주는 역사 교수야. 답변 마지막에는 답변과 연관된 이모티콘을 추가해줘."},
    {"role": "user", "content": "이순신이 활약한 전투는?"}
]

response = client.chat.completions.create(
    model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    messages= messages,
    max_tokens=300,
    temperature=0.0,
    top_p=0.9
)

print(response.choices[0].message.content)

## 메시지
시스템 메시지 뒤에 사용자와 어시스턴트 사이의 대화를 일련의 메시지로 포함시킬 수 있다.

In [ ]:
messages= [
    {"role": "system", "content": "너는 한국사 질문을 답변해주는 역사 교수야. 답변 마지막에는 답변과 연관된 이모티콘을 추가해줘."},
    {"role": "user", "content": "이순신이 활약한 전투는?"},
    {"role": "assistant", "content": "이순신이 활약한 전투 중에서 가장 유명한 것은 조선 시대의 해상 전투 중 하나인 명량 해전입니다."},
    {"role": "user", "content": "명량 해전의 성과를 알려줘."},
]

In [ ]:
response = client.chat.completions.create(
    model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"), 
    messages= messages,
    max_tokens=300,
    temperature=0.0,
    top_p=0.9
)

print(response.choices[0].message.content)

## 지식 추가하기
관련된 데이터나 지식을 시스템 메시지에 포함시켜 모델에 추가적인 맥락을 할당할 수 있다. 필요한 정보의 양이 적은 경우에는 시스템 메시지 안에 하드코딩해도 괜찮다. 정보의 양이 많으면 Embeddings API나 Azure AI Search 같은 제품을 사용하여 가장 연관성 높은 정보를 취득할 수 있다.

In [ ]:
system_message = """
너는 한국사 질문을 답변해주는 역사 교수야.

# 규칙
- 출처에서 답변을 찾을 수 없으면 '모르겠습니다'라고만 답변해줘.
- 출처에 없는 내용을 함부로 추론하지말 것.

문맥:
###
이의방: 이의방.pdf
이의방(李義方, 1121년~1175년 1월 19일(율리우스력1월 12일)(1174년 음력 12월 18일)은 고려의 무신이다. 본관은 전주(全州)이다. 
1170년(고려 의종 23년) 정중부, 이고와 함께 무신정변을 일으켜 응양용호군(鷹揚龍虎軍)의 중랑장(中郞將)에 임명되고,
무신정권을 수립한 뒤 대장군(大將軍) 전중감(殿中監) 겸 집주(執奏)에 임명되었다. 
좌승선(左承宣)으로 조위총의 난을 진압하던 중 정중부의 아들인 정균에 의해 피살되었다.
###

질문:
"""

messages= [
    {"role": "system", "content": system_message},
    {"role": "user", "content": "정중부, 이고와 함께 무신정변을 일으킨 인물의 이름을 알려줘."},
]

response = client.chat.completions.create(
    model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"), 
    messages= messages,
    max_tokens=300,
    temperature=0.0,
    top_p=0.9
)

print(response.choices[0].message.content)

## Chat Completion을 활용한 Few-shot Learning
모델에는 몇 가지 예시를 제공할 수 있다. Few-shot Learning 기법을 사용할 때는 프롬프트 형식이 약간 달라진다. 사용자와 어시스턴트 간에 주고 받은 일련의 메시지를 Few-shot 예시로서 프롬프트에 포함시키면 된다. 이 예시들을 활용하면 모델에 특정한 답변 형식을 지정하거나 기본 데이터를 학습시키는 것과 같은 효과를 얻을 수 있다.



In [ ]:
messages= [
    {"role": "system", "content": "너는 한국사 질문을 답변해주는 역사 교수야. 답변 마지막에는 답변과 연관된 이모티콘을 추가해줘."},
    {"role": "user", "content": "이순신이 활약한 전투는?"},
    {"role": "assistant", "content": "이순신이 활약한 전투 중에서 가장 유명한 것은 조선 시대의 해상 전투 중 하나인 명량 해전입니다.🚢"},
    {"role": "user", "content": "명량 해전의 성과를 알려줘."},
    {"role": "assistant", "content": "이순신 장군은 단 13척의 배로 왜군의 130여 척에 달하는 대규모 함대를 맞아 승리를 이끌었습니다.⚔️🌊"},
    {"role": "user", "content": "당시 조선 사람들의 생활상을 알려줘."},
    {"role": "assistant", "content": "왜군의 침입으로 많은 마을과 도시가 파괴되었고, 농경지가 황폐화되었습니다. 이로 인해 식량 부족과 경제적 어려움이 발생했습니다.🏚️🌾"},
    {"role": "user", "content": "전쟁의 결과는?"},
]

response = client.chat.completions.create(
    model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    messages= messages,
    max_tokens=300,
    temperature=0.0,
    top_p=0.9
)

print(response.choices[0].message.content)

## 모델별 토큰 계산법
OpenAI의 [tiktoken](https://github.com/openai/tiktoken) 라이브러리를 사용해서 토큰수를 계산할 수 있다.
tiktoken 라이브러리를 설치하려면 다음 커맨드를 실행한다.

`!pip install tiktoken --upgrade`

In [ ]:
import tiktoken

# "gpt-3.5-turbo-0613",
# "gpt-3.5-turbo-16k-0613",
# "gpt-4-0314",
# "gpt-4-32k-0314",
# "gpt-4-0613",
# "gpt-4-32k-0613",
model = "gpt-3.5-turbo-0613"
value = "이순신이 활약한 전투는?"

try:
    encoding = tiktoken.encoding_for_model(model)
except KeyError:
    print("Warning: model not found. Using cl100k_base encoding.")
    encoding = tiktoken.get_encoding("cl100k_base")

print("Tokens:",len(value))
print(encoding.encode(value))
